In [ ]:
run ../utils/preprocess.py

In [ ]:
import constants as C

import matplotlib.pyplot as plt

In [ ]:
def tables_from_category(category):
    with open('../../data/pickle_files/%s.pickle' % category, 'rb') as f:
        return pickle.load(f)

qa_table, reviews_table = tables_from_category(C.VIDEO_GAMES)

#Question with maximum length

In [ ]:
mask = qa_table['questionText'].str.len() == 20
result = qa_table.loc[mask]['questionText']
question = result.iloc[1]

In [ ]:
print question

In [ ]:
stopset_u = set(stopwords.words('english'))
stopset = set([str(sw) for sw in stopset_u])

tokens = nltk.word_tokenize(question)
tokens = [w.lower() for w in tokens if not w.lower() in stopset]

print len(tokens)

#Questions containing "Read More"

In [ ]:
mask = qa_table['answerText'].str.contains("Read More")
result = qa_table.loc[mask]['questionText']

In [ ]:
#Preprocess

In [ ]:
reviewsLengthList, questionsLengthList, answersLengthList = generate_length_lists(C.VIDEO_GAMES)

In [ ]:
df = get_raw_dataframe(C.VIDEO_GAMES)

In [ ]:
len(df)

In [ ]:
item = C.LENGTH_DICT[16]
C.JSON_DATA_PATH = '../../data/json_data'
C.INPUT_DATA_PATH = '../../data/input'

In [ ]:
filter_raw_data(item['Category'], item['ReviewLength'], item['QuestionLength'], item['AnswerLength'])

In [ ]:
pd = get_filter_dataframe(C.VIDEO_GAMES)

In [ ]:
pd.head()

In [ ]:
category = "Video_Games"
C.INPUT_DATA_PATH = '../../data/input'
with open('%s/train-%s.pickle' % (C.INPUT_DATA_PATH, category), 'rb') as f:
    train_data = pd.read_pickle(f)

In [ ]:
train_data.head()

In [ ]:
for questionsList in train_data['questionsList']:
    for question in questionsList:
        for answer in question['answers']:
                print(answer['text'])

In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import numpy as np
from operator import itemgetter, attrgetter
from torch.utils.data import Dataset, DataLoader

In [18]:
reviews1 = [[1,2], [1,2,3],[6,7,8]]
reviews2 = [[1], [1,2]]

question1 = [1,2,3]
question2 = [2,3,4,5]

answer1 = [1,3]
answer2 = [2]

data1 = [(answer1), (answer2)]
data2 = [(answer1, question1), (answer2, question2)]
data3 = [(answer1, question1, reviews1), (answer2, question2, reviews2)]

In [19]:
print(data1)
print(data2)
print(data3)

[[1, 3], [2]]
[([1, 3], [1, 2, 3]), ([2], [2, 3, 4, 5])]
[([1, 3], [1, 2, 3], [[1, 2], [1, 2, 3], [6, 7, 8]]), ([2], [2, 3, 4, 5], [[1], [1, 2]])]


In [20]:
a, q = zip(*data2)
print(a)
print(q)

([1, 3], [2])
([1, 2, 3], [2, 3, 4, 5])


In [56]:
class AmazonDataLoader(DataLoader):

    def sortByLength(self, item):
        if self.mode is "1":
            return len(item)
        
        elif self.mode is "2":
            assert(len(item) == 2)
            return len(item[0])
        
        elif self.mode is "3":
            assert(len(item) == 3)
            reviews = item[2]
            max_len = 0
            for review in reviews:
                max_len = max(max_len, len(review))
            return max_len

        
    def __init__(self, data, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        
        sorted(data, key=self.sortByLength, reverse=True)    
        self.data = data
        

    def create_packed_qa(self, batch_data):
        batch_data.sort(key=len, reverse=True)
        lengths = np.array([len(item) for item in batch_data])
        max_len = max(lengths)
        
        padded_data = np.array( [np.pad(item, (0,max_len-len(item)), 'constant') for item in batch_data] )
        padded_data = torch.from_numpy(padded_data)
        if torch.cuda.is_available():
            padded_data = padded_data.cuda()
        padded_data = Variable(padded_data)
        
        #pack_padded_sequence(padded_data, lengths)
        #Size is T*B T=MaxSeqLength, B=BatchSize
        return padded_data
    
    def create_packed_reviews(self, review_data):
        max_num_reviews = 0
        for reviews in review_data:
            max_num_reviews = max(max_num_reviews, len(reviews))
        
        data = []
        for i in range(max_num_reviews):
            batch_data = []
            for j in range(self.batch_size):
                reviews = review_data[j]
                if i < len(reviews):
                    batch_data.append(reviews[i])
                else:
                    batch_data.append([0])
            data.append(batch_data)
        
        padded_data = []
        for i in range(max_num_reviews):
            batch_data = data[i]
            max_len = 0
            for review in batch_data:
                max_len = max(max_len, len(review))
            
            padded_batch_data = np.array([np.pad(item, (0, max_len-len(item)), 'constant') for item in batch_data])
            padded_batch_data = torch.from_numpy(padded_batch_data)
            padded_data.append(padded_batch_data)
            
        return padded_data
        
    def __iter__(self):
        print(self.data)
        self.num_batches = len(self.data) // self.batch_size
        indices = np.arange(self.num_batches)
        np.random.shuffle(indices)

        for index in indices:
            start = index*self.batch_size
            end = (index+1)*self.batch_size
            
            batch_data = self.data[start:end]
            assert(self.batch_size == len(batch_data))
            
            if self.mode is "1":
                answers = batch_data
                packed_answers = self.create_packed_qa(answers)
                yield (packed_answers)
                
            elif self.mode is "2":
                answers, questions = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                yield (packed_answers, packed_questions)
            
            elif self.mode is "3":
                answers, questions, reviews = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                packed_reviews = self.create_packed_reviews(list(reviews))
                yield (packed_answers, packed_questions, packed_reviews)
                
    def __len__(self):
        return self.num_batches

In [57]:
data_loader = AmazonDataLoader(data3, "3", 2)

In [58]:
for batch_idx, data in enumerate(data_loader):
    print(batch_idx, data)

[([1, 3], [1, 2, 3], [[1, 2], [1, 2, 3], [6, 7, 8]]), ([2], [2, 3, 4, 5], [[1], [1, 2]])]
0 (
 1  3
 2  0
[torch.LongTensor of size (2,2)]
, 
 2  3  4  5
 1  2  3  0
[torch.LongTensor of size (2,4)]
, [
 1  2
 1  0
[torch.LongTensor of size (2,2)]
, 
 1  2  3
 1  2  0
[torch.LongTensor of size (2,3)]
, 
 6  7  8
 0  0  0
[torch.LongTensor of size (2,3)]
])


In [ ]:
def sortByMaxReviewLength(item):
    reviews = item[2]
    max_len = 0
    for review in reviews:
        max_len = max(max_len, review.shape[0])
    return max_len

for item in data3:
    assert(len(item) == 3)

sorted(data3, key=sortByMaxReviewLength, reverse=True)

In [ ]:
def sortByAnswerLength(item):
    return item[0].shape[0]

sorted(data2, key=sortByAnswerLength, reverse=True)

In [ ]:
answers = data1
print(answers)

answers, questions = zip(*data2)
print(data2)
print(answers)
print(questions)

answers, questions, reviews = zip(*data3)
print(data3)
print(answers)
print(questions)
print(reviews)

In [8]:
x = Variable(torch.randn(10, 10, 30))
lens = list(range(1, 11))
x = pack_padded_sequence(x, lens[::-1], batch_first=True)

In [9]:
x

PackedSequence(data=
-3.4826  0.6505  0.2313  ...  -0.4217 -1.1333  0.5225
 0.7663  0.1747 -1.2382  ...   1.3020 -0.6078  1.3379
 1.6234  1.1971  1.0253  ...   0.8074  0.5033  0.2730
          ...             ⋱             ...          
 0.7317  0.5685  0.3223  ...   0.4950  1.7678 -0.1372
-0.5602  0.9418 -0.2048  ...  -0.1512 -0.6764  0.9189
-0.2069 -0.3963 -0.1961  ...   0.7188  1.9704 -0.1466
[torch.FloatTensor of size (55,30)]
, batch_sizes=
 10
  9
  8
  7
  6
  5
  4
  3
  2
  1
[torch.LongTensor of size (10,)]
)

In [10]:
y = pad_packed_sequence(x)

In [11]:
y

(
 (0 ,.,.) = 
  -3.4826  0.6505  0.2313  ...  -0.4217 -1.1333  0.5225
   0.7663  0.1747 -1.2382  ...   1.3020 -0.6078  1.3379
   1.6234  1.1971  1.0253  ...   0.8074  0.5033  0.2730
            ...             ⋱             ...          
   1.8438 -0.5399  0.7004  ...  -1.6241 -2.0630  0.3565
  -0.2727 -0.5130  0.5228  ...  -0.3009  0.3447 -0.3301
   1.5795 -0.6392 -0.2103  ...   1.4226  1.4350 -1.3024
 
 (1 ,.,.) = 
  -0.0898 -1.7344 -1.4330  ...  -0.7859 -0.2026  1.6404
   1.4913 -1.1662  0.1572  ...   1.5091 -0.9296  0.7801
   1.3479  0.1117  0.7511  ...  -2.1069 -0.1227 -0.3189
            ...             ⋱             ...          
   0.5413 -1.7192  1.3961  ...  -0.9017 -0.2772 -0.4945
   1.8068 -1.0969 -2.0279  ...  -1.4663  0.8353  0.0754
   0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 
 (2 ,.,.) = 
  -1.0612  0.3370  0.7215  ...   0.1293 -0.0694 -1.4525
   0.1806  0.7922  0.4486  ...  -1.3753  0.4514  0.1990
   0.4137  1.0701  0.2034  ...   0.3928 -1.0111 -0.0325
   